In [1]:
import pandas as pd
import requests
import os
import plotly.offline as py
from plotly import figure_factory as ff
import ipywidgets as widgets
import plotly.graph_objs as go
from PIL import Image
import json
#Made for displays online aka. not buying the pay version
py.init_notebook_mode(connected=True)

In [2]:
Piserver = 'http://192.168.1.44:5000/'
dataService = pd.read_csv(Piserver)

w = widgets.Text(value='',description='File name',disabled=False)
w2 = widgets.Text(value='',description='Find file',disabled=False)
services=['Files','Buscar','Download', 'Upload']
opcionesServicios = widgets.SelectMultiple(options=list(services), description='Services')  
options=['Foto','Datos','Todo']
opciones = widgets.SelectMultiple(options=list(options),description='Type')

tablaServicios = ff.create_table(dataService)

In [3]:
#Connecton to list of files
data = pd.read_json(Piserver + 'files')
data.to_csv('data.csv',index=False )
df = pd.read_csv('data.csv')
table = ff.create_table(df)

py.iplot(tablaServicios, filename='tablaServicios')

In [5]:
def callService(opcionesServicios):
    if 'Files' in opcionesServicios:        
        py.iplot(table, filename='tablaArchivos')
        
    elif 'Buscar' in opcionesServicios:   
        display(w2) 
        w2.observe(busca,names='value')
        display(widgets.interactive(updateTabla,opciones=opciones))
        
    elif 'Download' in opcionesServicios:
        display(w)
        w.observe(downloadAnImage, names='value') 
        
def busca(change):    
    if w2.value in df['archivos'].unique():
        table =ff.create_table(df[df['archivos']== w2.value])        
        #Abre una pestaña y te muestra el archivo seleccionado
        py.iplot(table, filename='tablaArchivos.html') 

def updateTabla(opciones):        
    if 'Foto' in opciones:
        table =ff.create_table(df[df['direccion']== '{}/download/image/nameOfImage.extention'])
        py.iplot(table, filename='tablaArchivos')
        
    elif 'Datos' in opciones:
        table = ff.create_table(df[df['direccion']== '{}/download/data/archiveName.extention'] )        
        py.iplot(table,filename = 'tablaArchivos')
        
    else:
        table = ff.create_table(df)
        py.iplot(table, filename='tablaArchivos')      

def downloadAnImage(change):
    if w.value in df['archivos'].unique():
        table =ff.create_table(df[df['archivos']== w.value])
        #Abre una pestaña y te muestra el archivo seleccionado
        py.iplot(table, filename='tablaArchivos.html')   
        response = requests.get(Piserver + 'download/image/{}'.format(w.value),stream=True)
        filename = w.value
         #Te recorres todo lo que hay en la respuesta y lo escribes en el archivo abierto
        with open(filename, 'wb') as f:
            for chunk in response:
                f.write(chunk)        
        img = Image.open(w.value)
        print(w.value + ' downloaded')
        img.show()
        
        
#To display objects options           
widgets.interactive(callService,opcionesServicios=opcionesServicios)        

aW50ZXJhY3RpdmUoY2hpbGRyZW49KFNlbGVjdE11bHRpcGxlKGRlc2NyaXB0aW9uPXUnU2VydmljZXMnLCBpbmRleD0oMiwpLCBvcHRpb25zPSgnRmlsZXMnLCAnQnVzY2FyJywgJ0Rvd25sb2HigKY=
